# Parse NOC to ISO data from wikipedia
Because NOC in dataframe differentiate t.ex Soviet Union, EUN, and Russia, we prefer to use ISO to plot on map. For Soviet Union, EUN and Russia, they have the same country: Russia. 

We therefore parse noc to iso dataframe from wikipedia.
However, one question is to get red of the reference number in the country column.

We therefore apply the following method:
[Reference][link]: Pandas read_html() ignore superscripts and subscripts

[link]:https://stackoverflow.com/questions/64309887/pandas-read-html-ignore-superscripts-and-subscripts

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://simple.wikipedia.org/wiki/Comparison_of_IOC,_FIFA,_and_ISO_3166_country_codes'

soup = BeautifulSoup(requests.get(url).content, 'html.parser')

# remove <sup>
for sup in soup.select('sup'):
    sup.extract()

table = pd.read_html(str(soup))[0]
table.head()

,Flag,Country,IOC,FIFA,ISO
0,NaN,Afghanistan,AFG,AFG,AFG
1,NaN,Åland Islands,NaN,NaN,ALA
2,NaN,Albania,ALB,ALB,ALB
3,NaN,Algeria,ALG,ALG,DZA
4,NaN,American Samoa,ASA,ASA,ASM


In [2]:
# select relevant columns
table = table.iloc[:, [1, 2, 4]]
table = table.rename(columns={"IOC": "NOC"})
noc_iso = table[table["NOC"].notna()]
noc_iso.head()

,Country,NOC,ISO
0,Afghanistan,AFG,AFG
2,Albania,ALB,ALB
3,Algeria,ALG,DZA
4,American Samoa,ASA,ASM
5,Andorra,AND,AND


- add rows to dataframe
- where both URS(soviet union), EUN([Unified Team at the Olympics][link]) are the regions Russia

[link]: https://en.wikipedia.org/wiki/Unified_Team_at_the_Olympics

In [3]:
data = {'Country': ['Russia', 'Russia'],
	'NOC': ['URS', 'EUN'],
	'ISO': ['RUS', 'RUS']}
new_row = pd.DataFrame(data)
noc_iso = noc_iso.append(new_row, ignore_index=True)
noc_iso.tail()

,Country,NOC,ISO
202,Yemen,YEM,YEM
203,Zambia,ZAM,ZMB
204,Zimbabwe,ZIM,ZWE
205,Russia,URS,RUS
206,Russia,EUN,RUS


In [4]:
noc_iso.to_csv("data/noc_iso.csv")